# Image Classifier



---


Rede CNN com 4 camadas pra classificação de imagens.

Para versões do Keras >= 2 não utilizar padding='same' para strides != 1. De acordo com a documentação do Keras é incompatível. [ver aqui](https://keras.io/layers/convolutional/)

---

Para treinamento com classes binárias alterar os seguintes parâmetros:

* loss='categorical_crossentropy' para loss='binary_crossentropy'
* class_mode="categorical" para class_mode="binary"
* Na última "Dense" alterar o número de valores de saída pra "1", se for categorical deve ser o número de categorias possíveis



### Verificando configurações do TensorFlow

In [0]:
# Checando a versão do Tensorflow
import tensorflow
tensorflow.__version__

In [0]:
# Verificando o funcionamento da GPU
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

### Instando bibliotecas faltantes (uso do colab)

In [0]:
# Instalando biblioteca para plotar gráfico durante o treinamento
!pip install livelossplot
from livelossplot import PlotLossesKeras

### Variáveis para outup do treino

In [0]:
# Declarando variáveis para registros de saída
TRAINING_LOGS_FILE = "training_logs.csv"
MODEL_SUMMARY_FILE = "model_summary.txt"
TEST_FILE = "test_file.txt_v2"
MODEL_FILE = "model_three_categories.h5"
MODEL_TENSORFLOW = "model_three_categories.pb"

### Importando bibliotecas

In [0]:
# Bibliotecas para treinamento
%matplotlib inline
import numpy as np
import tensorflow as tf

# Necessário para preparação de dados
from keras.preprocessing.image import ImageDataGenerator

# Necessário para estruturação da rede
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import CSVLogger

# Necessário para conexão com Google Drive
from google.colab import drive, files

# Necessário para coleta e apresentação de métricas e resultados do modelo
from sklearn.metrics import confusion_matrix, accuracy_score
from mlxtend.plotting import plot_confusion_matrix
from sklearn import metrics
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Necessário para configuração do treino
from keras.callbacks import EarlyStopping

# Necessário para carregar um modelo Keras salvo
from keras.models import load_model

# Necessário necessária para converter um modelo Keras em um modelo Tensorflow
from keras import backend as K

### Definindo path a partir do Google Drive

In [0]:
path = ""
drive.mount('/content/drive/')
path = "/content/drive/My Drive/image_classifier/"
path_to_save_model_keras = path + "keras_model/" + MODEL_FILE 
path_to_save_model_tensorflow = path + "tensorflow_model/model/"  
training_data_dir = path + "dataset/train"
validation_data_dir = path + "dataset/val"
test_data_dir = path + "dataset/test"

  ### Hyperparâmetros

In [0]:
IMAGE_SIZE = 200
IMAGE_WIDTH, IMAGE_HEIGHT = IMAGE_SIZE, IMAGE_SIZE
EPOCHS = 100
BATCH_SIZE = 32
TEST_SIZE = 20

input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)

### Estrutura da Rede

In [0]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=3, strides=1, input_shape=input_shape, padding='same', activation='relu'))
model.add(Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))

model.add(Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))

model.add(Conv2D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(Conv2D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))

model.add(Conv2D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(Conv2D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(3))
model.add(Activation('softmax'))
    
model.compile(loss='categorical_crossentropy',
            optimizer=RMSprop(lr=0.0001),
            metrics=['accuracy'])

summary_path = path + MODEL_SUMMARY_FILE
with open(summary_path,"w") as fh:
    model.summary(print_fn=lambda line: fh.write(line + "\n"))

### Preparando Dados

In [0]:
# Aumentando Dados
training_data_generator = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True)
validation_data_generator = ImageDataGenerator(rescale=1./255)
test_data_generator = ImageDataGenerator(rescale=1./255)

# Preparando Dados
training_generator = training_data_generator.flow_from_directory(
    training_data_dir,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode="categorical")
validation_generator = validation_data_generator.flow_from_directory(
    validation_data_dir,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode="categorical")
test_generator = test_data_generator.flow_from_directory(
    test_data_dir,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=1,
    class_mode="categorical", 
    shuffle=False)

### Treinando

In [0]:

# Configurando o Callbak para salvar quando o loss parar de diminuir
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

training_logs_path = path + TRAINING_LOGS_FILE

# Treinamento
model.fit_generator(
    training_generator,
    steps_per_epoch=len(training_generator.filenames) // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=len(validation_generator.filenames) // BATCH_SIZE,
    callbacks=[PlotLossesKeras(), CSVLogger(training_logs_path,
                                            append=False,
                                            separator=","), early_stopping], 
    verbose=1)

# salvando modelo
model.save(path_to_save_model_keras)

### Carregando um modelo salvo

In [0]:
model = load_model(path_to_save_model_keras)
model.summary();

### Imprimindo a predição para cada imagem de teste utilizando o modelo Keras treinado


In [0]:
# Imprime a accuracy final do modelo
accuracy = accuracy_score(y_true, y_pred)
print('Acurácia final do modelo: ',  accuracy)

# Retorna o label da classes
class_names = list(test_generator.class_indices.keys())

# Retorna a predição probabilística das imagens de teste
probabilities = model.predict_generator(test_generator)

# Retorna o valor real das imagens utilizadas na previsão
real_value = test_generator.classes

# Configura a fonte do título da image
plt.title('The title with font size: 20, and font:monospace')


for index, probability in enumerate(probabilities):
    
    # Obtendo o path da imagem no Drive
    image_path = test_data_dir + "/" +test_generator.filenames[index]
    
    # Lê a imagem a pardir do diretório do Google Drive
    img = mpimg.imread(image_path)
    
    # Obtém o indice do maior valor da predição
    max_index = np.argmax(probability)
    
    # Salva os dados do teste de predição em um arquivo .txt
    with open(TEST_FILE,"a") as fh:
      
        fh.write(str(probability[max_index]) + " for: " + image_path + "\n")
        
    #plt.imshow(img)
    
    color  = 'g' if class_names[max_index] == class_names[real_value[index]] else 'r'
       
    plt.title(str(index) + ". " + "%.2f" % (probability[max_index]*100) + "%" + " - " + "Predito: " + class_names[max_index] + " | Esperado: " + class_names[real_value[index]], fontsize=14, color=color)

    plt.show()

0.8913443830570903


### Matriz de Confusão

In [0]:
class_names = list(test_generator.class_indices.keys())
c_matrix = confusion_matrix(y_true, y_pred)
classes = [''] + class_names

fig, ax = plot_confusion_matrix(conf_mat=c_matrix,
                                colorbar=True,
                                figsize=(8,8),
                                cmap='GnBu',
                                show_absolute=False,
                                show_normed=True)


ax.xaxis.set_ticklabels(classes, fontsize=12); 
ax.yaxis.set_ticklabels(classes, fontsize=12);
ax.set_title('Confusion Matrix', fontsize=16);


plt.show();

### Convertendo um modelo do Keras para Tensorflow

In [0]:

tf.keras.backend.set_learning_phase(0)  # Ignore dropout at inference

export_path = path_to_save_model_tensorflow


with tf.keras.backend.get_session() as sess:
    sess.run(tf.global_variables_initializer())
    model = load_model(path_to_save_model_keras)
    tf.saved_model.simple_save(
        sess,
        export_path,
        inputs={'input_image': model.input},
        outputs={t.name: t for t in model.outputs})